# Play around with bokeh

In [27]:
import numpy as np
from bokeh.plotting import figure,show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource,CDSView, GroupFilter
from bokeh.transform import factor_cmap, factor_mark,CategoricalMarkerMapper
from bokeh.models import HoverTool
from bokeh.layouts import gridplot

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
p = figure()
N = 1000
x = np.random.random(size = N)
y = np.random.random(size = N)

p.circle(x = x,y = y)

GlyphRenderer(id='1037', ...)

In [4]:
show(p)

In [5]:
import pandas as pd

In [6]:
df_cars = pd.read_csv('cars_publish.csv',index_col=0)

In [7]:
df_cars.head()

,Biltyp,Bränsle,CO2-utsläpp,Drivhjul,Färg,Hästkrafter,I trafik,Miltal,Modell,Modellår,Märke,Tillverkningsår,Växellåda,header,href,place,price,car type
86506490,Kombi,Diesel,123 g/km,Tvåhjulsdriven,Vit,89.0,2016-05-30,6985.0,KANGOO,2016.0,Renault,2015.0,Manuell,Renault Kangoo 1.5 dCi Maxi skåp (90hk) -16,https://www.blocket.se/vasternorrland/Renault_...,Västernorrland,68000.0,kangoo
86463241,Kombi,Diesel,-,Tvåhjulsdriven,Vit,89.0,2013-07-25,10600.0,W KANGOO,2013.0,Renault,2013.0,Manuell,Renault Kangoo Express Maxi 1.5 dCi 90hk 5sit -13,https://www.blocket.se/jamtland/Renault_Kangoo...,Jämtland,97000.0,kangoo
86219343,Kombi,Diesel,-,Tvåhjulsdriven,Vit,84.0,2006-12-18,18249.5,FC KANGOO,2007.0,Renault,2007.0,Manuell,Kangoo 1.5dci Skåp Drag Kamrem bytt -07,https://www.blocket.se/jamtland/Kangoo_1_5dci_...,Jämtland,8500.0,kangoo
86211627,Kombi,Diesel,-,Tvåhjulsdriven,Ljusgrå (Lgrå),81.0,2004-01-28,20500.0,KANGOO FC08BF,2004.0,Renault,2004.0,Manuell,Renault Kangoo Diesel( Trasig Växelåda ) -04,https://www.blocket.se/sodermanland/Renault_Ka...,Södermanland,2300.0,kangoo
83751407,Kombi,Diesel,-,Tvåhjulsdriven,Vit,89.0,2014-04-02,7515.0,W KANGOO,2014.0,Renault,2014.0,Manuell,"Renault Kangoo 1,5 Dci 90 Hk Style Family 5d -14",https://www.blocket.se/dalarna/Renault_Kangoo_...,Dalarna,119900.0,kangoo


In [8]:
list(df_cars.columns)

['Biltyp',
 'Bränsle',
 'CO2-utsläpp',
 'Drivhjul',
 'Färg',
 'Hästkrafter',
 'I trafik',
 'Miltal',
 'Modell',
 'Modellår',
 'Märke',
 'Tillverkningsår',
 'Växellåda',
 'header',
 'href',
 'place',
 'price',
 'car type']

In [30]:
source = ColumnDataSource(data=df_cars)
#source = ColumnDataSource(data=df_cars.groupby(by = 'car type'))


In [31]:
hover = HoverTool(
        tooltips=[
            ("Modell", "@Modell"),
            ("price", "@price"),
            
                    ]
    )

# Specify the tools
toolList = ['lasso_select', 'tap', 'reset', 'save',hover]

markers = ['circle']

p = figure(title = 'Cars',tools = toolList)

x_column = 'Tillverkningsår'
y_column = 'price'
p.scatter(x=x_column, y=y_column, source=source,
          legend="car type",
          marker=factor_mark('car type',markers = markers,factors = df_cars['car type'].unique()),
          color=factor_cmap('car type',palette = 'Category10_3',factors = df_cars['car type'].unique()),
          size = 8,alpha = 0.25)

p.xaxis.axis_label = x_column
p.yaxis.axis_label = y_column
p.legend.click_policy = 'hide'

p2 = figure(title = 'Cars',tools = toolList)

x_column = 'Miltal'
y_column = 'price'
p2.scatter(x=x_column, y=y_column, source=source,
          legend="car type",
          marker=factor_mark('car type',markers = markers,factors = df_cars['car type'].unique()),
          color=factor_cmap('car type',palette = 'Category10_3',factors = df_cars['car type'].unique()),
          size = 8,alpha = 0.25)

p2.xaxis.axis_label = x_column
p2.yaxis.axis_label = y_column
p2.legend.click_policy = 'hide'


# Create layout
grid = gridplot([[p, p2]])

In [32]:
show(grid)

In [54]:
# Specify the tools
toolList = ['lasso_select', 'tap', 'reset', 'save','box_zoom','pan','wheel_zoom',hover]

markers = ['circle']

kangoo_filters = [GroupFilter(column_name='car type', group='kangoo')]
berlingo_filters = [GroupFilter(column_name='car type', group='berlingo')]
caddy_filters = [GroupFilter(column_name='car type', group='caddy')]

kangoo_view = CDSView(source=source,
                      filters=kangoo_filters)

berlingo_view = CDSView(source=source,
                      filters=berlingo_filters)

caddy_view = CDSView(source=source,
                      filters=caddy_filters)

p = figure(title = 'Cars',tools = toolList)

x_column = 'Tillverkningsår'
y_column = 'price'

p.scatter(x=x_column, y=y_column, view = kangoo_view,source = source,
          legend="kangoo",
          color=factor_cmap('car type',palette = 'Category10_3',factors = df_cars['car type'].unique()),
          size = 8,alpha = 0.25)

p.scatter(x=x_column, y=y_column, view = berlingo_view,source = source,
          legend="berlingo",
          color=factor_cmap('car type',palette = 'Category10_3',factors = df_cars['car type'].unique()),
          size = 8,alpha = 0.25)

p.scatter(x=x_column, y=y_column, view = caddy_view,source = source,
          legend="caddy",
          color=factor_cmap('car type',palette = 'Category10_3',factors = df_cars['car type'].unique()),
          size = 8,alpha = 0.25)

p.xaxis.axis_label = x_column
p.yaxis.axis_label = y_column
p.legend.click_policy = 'hide'


# Create layout
grid = gridplot([[p]])

In [55]:
show(grid)